In [ ]:
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import bambi as bmb
import xarray as xr
import random
%load_ext rpy2.ipython


In [ ]:
%%R
library(tidyverse)

In [ ]:
%%R
b_species <- read_delim('../results/model/Nested_model/b_14_12_2023', delim = '\t')
h0 <- read_delim('../results/model/Nested_model/h0_14_12_2023', delim = '\t')
h1 <- read_delim('../results/model/Nested_model/h1_14_12_2023', delim = '\t')
Ne <- read_csv('../results/model/Nested_model/Reduced_Ne_Pi_recomb.csv') %>% select(full_species,NE_MEDIAN) %>% distinct(full_species, .keep_all=TRUE)

In [ ]:
%%R
b_species <- b_species %>% 
    group_by(Species,draw) %>% reframe(
    b = mean(b)) %>%
    group_by(Species) %>% reframe(
    sd = sd(b),
    b = mean(b))
b_species

In [ ]:
%%R
merged_data <- merge(b_species, Ne, by.x = "Species", by.y = "full_species", all.x = TRUE) 
merged_data <- merged_data %>% mutate(
    scaled_ne = scale(merged_data$NE_MEDIAN,center=TRUE,scale=TRUE))

In [ ]:
%%R
fig1 <- merged_data %>% ggplot(aes(x=NE_MEDIAN,y=b))+
geom_point(aes(col=NE_MEDIAN))+
geom_line(aes(x=NE_MEDIAN, y=(mean(h0$h0))+(mean(h1$h1)*scaled_ne)), lty = 2)+
geom_errorbar(aes(ymin=b-sd, ymax=b+sd, col = NE_MEDIAN), width=.2,
                 position=position_dodge(.9)) +
scale_color_continuous(low = "#ffff33", high = "#e41a1c") +
theme(plot.margin = margin(t = 0, r = 0, b = 0, l = 0.2, unit = "cm"), legend.key.size = unit(0.5,"line"))+
 ylab("Slope")+
 xlab('Median Effecive population size')+
theme_classic()+
theme(legend.position = "none")

In [ ]:
%%R
ggsave('test.pdf')